In [ ]:
from trino.dbapi import connect, Cursor
import pandas as pd

In [ ]:
def get_trino_queries_df(cursor: Cursor) -> pd.DataFrame:
    cur.execute("SELECT * FROM system.runtime.queries")
    rows = cur.fetchall()
    
    columns = [
        'query_id',
        'state',
        'user',
        'source',
        'query',
        'resource_group_id',
        'queued_time_ms',
        'analysis_time_ms',
        'planning_time_ms',
        'created',
        'started',
        'last_heartbeat',
        'end',
        'error_type',
        'error_code'
    ]
    df = pd.DataFrame(rows, columns=columns)
    df.drop(columns=['resource_group_id'], inplace=True)       
    for col in [
        'created',
        'started',
        'last_heartbeat',
        'end',
    ]:
        try:
            df[col] = pd.to_datetime(df[col], errors='coerce')
            df[col] = df[col].dt.tz_convert(None) if df[col].dt.tz is not None else df[col]
        except AttributeError:
            pass
    
    df.sort_values('created', inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [ ]:
HOST = '127.0.0.1'
PORT = 21123
USERNAME = 'analyzer'


conn = connect(
    host=HOST,
    port=PORT,
    user=USERNAME,
)
cur = conn.cursor()

df = get_trino_queries_df(cur)
# df.to_excel('trino_logs.xlsx', index=False)

In [ ]:
df.tail()